# Face Mask Detection

In [1]:
import os
import tensorflow as tf
import cv2
import numpy as np

import warnings
warnings.filterwarnings("ignore")

In [2]:
model = tf.keras.models.load_model("model/facemask_model.h5")

2022-07-22 09:16:33.789123: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
labels_dict = {0:'Without Mask',1:'With Mask'}
color_dict = {0: (0,0,255),
              1:(0,255,0)}

size = 4
webcam = cv2.VideoCapture(0) 

face_cascade_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [6]:
while True:
    (rval, im) = webcam.read()
    im=cv2.flip(im,1,1) #Flip to act as a mirror

    # Resize the image to speed up detection
    mini = cv2.resize(im, (im.shape[1] // size, im.shape[0] // size))

    # detect MultiScale / faces 
    faces = face_cascade_classifier.detectMultiScale(mini)

    # Draw rectangles around each face
    for f in faces:
        (x, y, w, h) = [v * size for v in f] #Scale the shapesize backup
        #Save just the rectangle faces in SubRecFaces
        face_img = im[y:y+h, x:x+w]
        resized = cv2.resize(face_img,(150,150))
        normalized = resized / 255.0
        reshaped = np.reshape(normalized,(1,150,150,3))
        reshaped = np.vstack([reshaped])
        result=model.predict(reshaped)
        
        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(im,(x,y),(x + w,y + h),color_dict[label],2)
        cv2.rectangle(im,(x, y - 40),(x+w,y),color_dict[label],-1)
        cv2.putText(im, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
    # Show the image
    cv2.imshow('LIVE', im)
    key = cv2.waitKey(10)
    # if Esc key is press then break out of the loop 
    if key == 27: #The Esc key
        break

# Stop video
webcam.release()

# Close all started windows
cv2.destroyAllWindows()